In [1]:
import arcpy
import os
import time

def process_mode(mode):
    start_time = time.time()

    # Set the workspace
    workspace = rf"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\MyPwoDSTPProject\MyPwoDSTPProject.gdb"
    arcpy.env.workspace = workspace
    arcpy.env.overwriteOutput = True

    ser_data_dir = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\ServiceArea_Tbig_Tsm\SerArea_{mode}"
    services = {
        "serv1": os.path.join(ser_data_dir, f"{mode}_Orig_serTbig.shp"),
        "serv2": os.path.join(ser_data_dir, f"{mode}_Destn_SerTbig.shp"),
        "serv3": os.path.join(ser_data_dir, f"{mode}_Orig_serTsm.shp"),
        "serv4": os.path.join(ser_data_dir, f"{mode}_Destn_SerTsm.shp")
    }

    output_folder = fr"C:\Users\kcmeg\Box\Paper to study for 10 sept\Travel demand modelling\Space-Time-Prism-Model\PwoD STP\PPA\{mode}_PPA"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)

    all_outputs = []
    all_ids = set()
    for service in services.values():
        all_ids.update([row[0] for row in arcpy.da.SearchCursor(service, 'hptripid')])

    feature_layers = {}
    for key, service in services.items():
        feature_layers[key] = arcpy.management.MakeFeatureLayer(service)

    for trip_id in all_ids:
        where_clause = f"hptripid = '{trip_id}'"
        for key, layer in feature_layers.items():
            arcpy.management.SelectLayerByAttribute(layer, "NEW_SELECTION", where_clause)

        intersect_temp1 = "in_memory\\IntersectTemp1"
        arcpy.analysis.Intersect([feature_layers['serv1'], feature_layers['serv2']], intersect_temp1)

        union_temp = "in_memory\\UnionTemp"
        arcpy.analysis.Union([feature_layers['serv3'], feature_layers['serv4']], union_temp)

        output_filename = arcpy.ValidateTableName(f"Result_{mode}_{trip_id}", output_folder)
        final_output = arcpy.CreateUniqueName(output_filename, output_folder)
        arcpy.analysis.Intersect([intersect_temp1, union_temp], final_output) 

        all_outputs.append(final_output)
        arcpy.management.Delete(intersect_temp1)
        arcpy.management.Delete(union_temp)

    merged_output = f"{output_folder}\\Merged_Temp"
    arcpy.management.Merge(all_outputs, merged_output)

    dissolved_output = f"{output_folder}\\PPA_{mode}"
    arcpy.management.Dissolve(merged_output, dissolved_output, "hptripid")

    arcpy.management.AddField(dissolved_output, "Shape_Leng", "DOUBLE")
    arcpy.management.AddField(dissolved_output, "Shape_Area", "DOUBLE")

    arcpy.management.CalculateField(dissolved_output, "Shape_Leng", "!shape.length!", "PYTHON3")
    arcpy.management.CalculateField(dissolved_output, "Shape_Area", "!shape.area!", "PYTHON3")

    for layer in feature_layers.values():
        arcpy.management.Delete(layer)
    for output in all_outputs:
        arcpy.management.Delete(output)
    arcpy.management.Delete(merged_output)

    print(f"Process completed successfully for {mode}.")
    processing_time = time.time() - start_time
    print(f"The script took {processing_time} seconds to complete for {mode}.")

def Model():
    modes = ["Bike", "Walk", "Transit", "Drive"]
    for mode in modes:
        process_mode(mode)

if __name__ == '__main__':
    Model()

Process completed successfully for Bike.
The script took 22.91772985458374 seconds to complete for Bike.
